# Stock Model Comparisons

In [10]:
import torch
import torch.nn as nn

## STOCKNN Model

In [11]:
class StockNN(nn.Module):
    def __init__(self, input_dim):
        super(StockNN, self).__init__()
        self.model = nn.Sequential(nn.Linear(input_dim, 20), nn.ReLU(),
                                   nn.Linear(20, 20), nn.ReLU(),
                                   nn.Linear(20, 20), nn.ReLU(),
                                   nn.Linear(20, 20), nn.ReLU(),
                                   nn.Linear(20, 20), nn.ReLU(),
                                   nn.Linear(20, 1))
    def forward(self, x):
        return self.model(x)

input_dim = 10

# Initialize model
model = StockNN(input_dim)

# Load saved model weights
model_path = 'STOCKNN.pth'
model.load_state_dict(torch.load(model_path, weights_only=True))

# Set model to evaluation mode
model.eval()

# Display model architecture
print("Model Architecture:\n", model)

# Display model weights
print("\nModel Weights:")
for name, param in model.named_parameters():
    print(f"{name}: {param.shape}\n{param}\n")

# Display model biases
print("\nModel Biases:")
for name, param in model.named_parameters():
    if 'bias' in name:
        print(f"{name}: {param.shape}\n{param}\n")

Model Architecture:
 StockNN(
  (model): Sequential(
    (0): Linear(in_features=10, out_features=20, bias=True)
    (1): ReLU()
    (2): Linear(in_features=20, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=20, bias=True)
    (5): ReLU()
    (6): Linear(in_features=20, out_features=20, bias=True)
    (7): ReLU()
    (8): Linear(in_features=20, out_features=20, bias=True)
    (9): ReLU()
    (10): Linear(in_features=20, out_features=1, bias=True)
  )
)

Model Weights:
model.0.weight: torch.Size([20, 10])
Parameter containing:
tensor([[ 0.1821,  0.2395, -0.0304,  0.2790, -0.1605,  0.0213, -0.1826,  0.1262,
          0.2166, -0.2836],
        [ 0.3421, -0.0248,  0.1187, -0.0474,  0.0825, -0.1234,  0.2047,  0.0683,
         -0.1934,  0.0074],
        [-0.1662, -0.2215, -0.1215,  0.1409, -0.3489, -0.1712, -0.1643, -0.0907,
          0.1047, -0.2760],
        [ 0.1693, -0.1965,  0.3936,  0.0578, -0.0635,  0.2061,  0.0744,  0.1145,
         -0.0296, 

## ADVANCEDSTOCKNN Model

In [12]:
class ResidualBlock(nn.Module):
    def __init__(self, dim, dropout_p=0.1):
        super(ResidualBlock, self).__init__()
        self.fc1 = nn.Linear(dim, dim)
        self.bn1 = nn.BatchNorm1d(dim)
        self.act = nn.SELU()
        self.drop = nn.AlphaDropout(dropout_p)
        self.fc2 = nn.Linear(dim, dim)
        self.bn2 = nn.BatchNorm1d(dim)
    def forward(self, x):
        residual = x
        out = self.fc1(x)
        out = self.bn1(out)
        out = self.act(out)
        out = self.drop(out)
        out = self.fc2(out)
        out = self.bn2(out)
        out = self.act(out)
        out = self.drop(out)
        return out + residual  # Skip connection

class AdvancedStockNN(nn.Module):
    def __init__(self, input_dim):
        super(AdvancedStockNN, self).__init__()
        # First layer: project from input_dim to a hidden size (we choose 16)
        self.fc1 = nn.Linear(input_dim, 16)
        self.bn1 = nn.BatchNorm1d(16)
        self.act = nn.SELU()
        self.drop = nn.AlphaDropout(0.1)
        # Two residual blocks in sequence
        self.resblock1 = ResidualBlock(16, dropout_p=0.1)
        self.resblock2 = ResidualBlock(16, dropout_p=0.1)
        # Final output layer
        self.fc_out = nn.Linear(16, 1)
    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.resblock1(x)
        x = self.resblock2(x)
        x = self.fc_out(x)
        return x
    
input_dim = 10

# Initialize model
advanced_model = AdvancedStockNN(input_dim)

# Load saved model weights
model_path = 'ADVANCED_STOCKNN.pth'
advanced_model.load_state_dict(torch.load(model_path, weights_only=True))

# Set model to evaluation mode
advanced_model.eval()

# Display model architecture
print("Model Architecture:\n", advanced_model)

# Display model weights
print("\nModel Weights:")
for name, param in advanced_model.named_parameters():
    print(f"{name}: {param.shape}\n{param}\n")

# Display model biases
print("\nModel Biases:")
for name, param in advanced_model.named_parameters():
    if 'bias' in name:
        print(f"{name}: {param.shape}\n{param}\n")

Model Architecture:
 AdvancedStockNN(
  (fc1): Linear(in_features=10, out_features=16, bias=True)
  (bn1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act): SELU()
  (drop): AlphaDropout(p=0.1, inplace=False)
  (resblock1): ResidualBlock(
    (fc1): Linear(in_features=16, out_features=16, bias=True)
    (bn1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act): SELU()
    (drop): AlphaDropout(p=0.1, inplace=False)
    (fc2): Linear(in_features=16, out_features=16, bias=True)
    (bn2): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (resblock2): ResidualBlock(
    (fc1): Linear(in_features=16, out_features=16, bias=True)
    (bn1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act): SELU()
    (drop): AlphaDropout(p=0.1, inplace=False)
    (fc2): Linear(in_features=16, out_features=16, bias=True)
    (bn2): BatchNorm1d(16, eps=1e-05,